##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Предсказывай цены на недвижимость: регрессия

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Читай на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запусти в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучай код на GitHub</a>
  </td>
</table>

Говоря о задачах *регрессии*, мы преследуем цель дать прогноз какого-либо значения, например цену или вероятность. Это уже совсем другая задача если сравненить с *классификацией*, где нужно предсказать конкретный класс или категорию (например, яблоко или апельсин на картинке).

В этом уроке мы построим модель, которая предсказывает медианную стоимость жилого дома в пригороде Бостона середины 1970-х. Чтобы это сделать, нам потребуются также некоторые данные об этом пригороде, например уровень преступности и налог на недвижимость.

В этом примере используется `tf.keras` API, подробнее [смотри здесь](https://www.tensorflow.org/guide/keras).

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## Загружаем данные с ценами на жилье Бостоне

Этот [датасет](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) доступен сразу в TensorFlow. Скачаем его и перемешаем данные:

In [ ]:
boston_housing = keras.datasets.boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# Перемешиваем данные в тренировочном сете
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

### Примеры и параметры данных

Этот датасет намного меньше других, с которыми работали ранее: в нем всего 506 записей, из которых 404 предназачены для обучения, а 102 - для проверки:

In [ ]:
print("Training set: {}".format(train_data.shape))  # 404 записи, 13 параметра
print("Testing set:  {}".format(test_data.shape))   # 102 записи, 13 параметра

Датасет содержит 13 разных параметров:

1.   Уровень преступности на душу населения
2.   Доля жилищных земель, районы свыше 25,000 кв. футов
3.   Доля нерозничных предприятий на район
4.   Переменная Charles River (равна 1 если земля имеет выход к воде; 0 - в противном случае)
5.   Концентрация выбросов азота
6.   Среднее количество комнат в доме
7.   Доля жилых домов, построенных до 1940 года
8.   Дистанция до пяти центров занятости Бостона
9.   Индекс доступности к дорожным автомагистралям
10.  Налоговая ставка на жилую недвижимость на $10,000
11.  Соотношение количества учеников и учителей на район
12.  1000 * (Bk - 0.63) ** 2, где Bk - доля афроамериканцев на район
13.  Процент бедности населения

Каждый из указанных параметров сохранен в разном формате. Некоторые параметры представлены пропорцией между 0 и 1, другие измеряются в диапазоне от 1 до 12, какие-то от 0 до 100, и так далее. Это наиболее частый случай когда мы имеем дело с реальными данными, и понимание того как анализировать и подготавливать такие данные является очень важным навыком.

Важный момент:

Как разработчик модели, подумай как эти данные могут быть использованы, а также какие потенциальные преимущества и возможный вред могут появиться в результате использования предсказаний модели. Модель наподобие этой может усилить общественные предубеждения и социальное неравенство. Является ли параметр, относящийся к решаемой проблеме, источником предубеждений? 

Прочитай про [справедливость в машинном обучении](https://developers.google.com/machine-learning/fairness-overview/).

In [ ]:
print(train_data[0])  # Отображает примеры параметров, обрати внимание на разный формат данных

Давай воспользуемся библиотекой [pandas](https://pandas.pydata.org) для отображения первых нескольких рядов датасета в виде понятной таблицы:

In [ ]:
import pandas as pd

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(train_data, columns=column_names)
df.head()

### Цены

Стоимость домов в нашем датасете указана как *label* в тысячах долларов (обрати внимание, что цены середины 70-х):

In [ ]:
print(train_labels[0:10])  # Выводит на экран первые 10 записей

## Нормализуем параметры

Рекомендуется нормализовывать параметры которые используют различные форматы и диапазоны значений. Для каждого параметра вычтем среднее значение и разделим на среднеквадратическое отклонение:

In [ ]:
# Проверочные данные *не используются* когда рассчитывается среднее значение и квадратическое отклонение

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

print(train_data[0])  # Отобразим первую запись нормализованных данных

Несмотря на то, что модель *может* стремиться к пределу без нормализации параметров, это делает обучение более сложным и получившаяся модель может быть более зависима от выбранных единиц измерения.

## Создаем модель

Давай построим нашу модель. Мы будем использовать `Sequential` (пер. "Последовательную") модель, состоящую из двух полносвязных скрытых слоев, а также последнего слоя, который будет возвращать одно значение. Все шаги построения модели мы оформим в функцию `build_model`, потому что позже мы создадим еще одну модель.

In [ ]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(train_data.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

## Обучаем модель

Мы будем обучать модель в течение 500 эпох и записывать точность тренировки и проверки в объект `history`.

In [ ]:
# Покажем готовность процесса обучения: после каждой завершенной эпохи будет появляться точка
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

# Сохраняем статистику обучения
history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

Теперь построим график, чтобы визуализировать прогресс обучения модели при помощи сохраненной статистики из `history`. Это поможет нам определить как долго нужно обучать модель до того, как модель *перестанет улучшать* показатели точности.

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [1000$]')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 5])

plot_history(history)

Эта схема показывает, что после 200 эпох наша модель улучшается совсем незначительно в процессе обучения. Давай обновим метод `model.fit` чтобы автоматически прекращать обучение как только показатель проверки *Val loss* не улучшается. Для этого мы используем функцию обратного вызова *callback*, которая проверяет показатели обучения после каждой эпохи. Если после определенного количество эпох нет никаких улучшений, то функиция остановит его.

Читай больше про функцию *callback* [здесь](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [ ]:
model = build_model()

# Параметр patience определяет количество эпох, которые можно пропустить без улучшений
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])

plot_history(history)

График показывает, что средняя стоимость ошибки \\$2,500 долларов. Это хорошо или плохо? Ну, \$2,500 не такая уж и большая сумма, если учитывать, что другие ярлыки имеют цены около $15,000.

Давай посмотрим, как справится наша новая модель на данных для проверки:

In [ ]:
[loss, mae] = model.evaluate(test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

## Делаем предсказания

Наконец давай предскажем цены на жилье, используя данные из проверочного сета:

In [ ]:
test_predictions = model.predict(test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [1000$]')
plt.ylabel('Predictions [1000$]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-100, 100], [-100, 100])


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [1000$]")
_ = plt.ylabel("Count")

## Заключение

Это руководство познакомило тебя с несколькими способами решения задач регрессии.

Ключевые моменты урока:

* Mean Squared Error (MSE) (пер. "Среднеквадратическая ошибка") - это распространенная функция потерь, используемая для решения задач регрессии (отличная от задач классификации)
* Показатели оценки модели для регрессии отличаются от используемых в классификации
* Когда входные данные имеют параметры в разных форматах, каждый параметр должен быть нормализован
* Если данных для обучения немного, используй небольшую сеть из нескольких скрытых слоев. Это поможет избежать переобучения
* Используй метод ранней остановки - это очень полезная техника для избежания переобучения